In [6]:
%pip install numpy pandas scipy uproot

UsageError: Line magic function `%pip3` not found.


In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import root_scalar
# import struct
import uproot
import paramiko
from getpass import getpass

# === CONFIG ===
INFILE = "/scratch/users/eeganr/june23output/output2.root"
NUM_VOLIDS = 6
cont_magnitude = 1e-5
num_TOF_bins = 9
TOF_bin_width = 29
sigma_TOF = 60
num_iterations = 2
num_subsets = 8

total_time = 1 # total sim time (s)
CYCLE = 1.6e-9  # clock cycle (s)
TAU = 3 * CYCLE  # coincidence window (s)
DELAY = 10 * CYCLE  # delay for DW estimate (s)
num_detectors = 48 * 48

In [2]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('login.sherlock.stanford.edu', 22, 'eeganr', getpass("Password: "))
key_auth = str(ssh.get_transport())
if 'awaiting auth' in key_auth:
    (ssh.get_transport()).auth_interactive_dumb('eeganr')

Duo two-factor login for eeganr

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-7276
 2. Phone call to XXX-XXX-7276
 3. SMS passcodes to XXX-XXX-7276

Passcode or option (1-3): 

In [3]:
sftp_client = ssh.open_sftp()
remote_file = sftp_client.open(INFILE)
remote_file.prefetch()

In [4]:
# Read in data from ROOT files

def get_all_vals(file, name):
    num = max([int(i.split(';')[1]) for i in file.keys() if i.split(';')[0] == name])
    return file[f'{name};{num}']

with uproot.open(remote_file) as file:
    singles_tree = get_all_vals(file, 'Singles')
    coincidence_tree = get_all_vals(file, 'Coincidences')

    singles = pd.DataFrame({
        "time": singles_tree["time"].array(library="np"),
        "detector": singles_tree["crystalID"].array(library="np"),
        "source": list(map(tuple, np.stack((singles_tree["sourcePosX"].array(library="np"), 
                            singles_tree["sourcePosY"].array(library="np"), 
                            singles_tree["sourcePosZ"].array(library="np")), axis=-1))),
        "energy": singles_tree["energy"].array(library="np"),
    })
    coincidences = pd.DataFrame({
        "time1": coincidence_tree["time1"].array(library="np"),
        "time2": coincidence_tree["time2"].array(library="np"),
        "detector1": coincidence_tree["crystalID1"].array(library="np"),
        "detector2": coincidence_tree["crystalID2"].array(library="np"),
        "source1": list(map(tuple, np.stack((coincidence_tree["sourcePosX1"].array(library="np"), 
                            coincidence_tree["sourcePosY1"].array(library="np"), 
                            coincidence_tree["sourcePosZ1"].array(library="np")), axis=-1))),
        "source2": list(map(tuple, np.stack((coincidence_tree["sourcePosX2"].array(library="np"), 
                            coincidence_tree["sourcePosY2"].array(library="np"), 
                            coincidence_tree["sourcePosZ2"].array(library="np")), axis=-1))),
    })
    coincidences['true'] = coincidences['source1'] == coincidences['source2']

In [5]:
# singles = singles[singles['energy'] > 0.450].reset_index()
# singles = singles[singles['energy'] < 0.750].reset_index()

In [6]:
coins = []
window_start = 0
possible_coincidence = []
for i in range(0, len(singles['time']) - 1):
    if singles['time'][i] - window_start >= TAU:
        # process any previous coincidences
        if len(possible_coincidence) >= 2:
            # If there are at least 2 singles in the window, we have a possible coincidence
            if len(possible_coincidence) > 2:
                energies = [singles['energy'][j] for j in possible_coincidence]
                energies = dict(zip(energies, possible_coincidence))
                true_coinc = (energies.pop(max(energies)), energies.pop(max(energies)))
                # Take just the two highest energies
                # takeWinnerOfGoods policy
            else:
                true_coinc = tuple(possible_coincidence)
            coins.append(true_coinc)

        # Reset the window
        possible_coincidence = [i]
        window_start = singles['time'][i]
    elif singles['time'][i] - window_start < TAU:
        possible_coincidence.append(i)  # Add to coincidence list
coinci = pd.DataFrame()
coinci['time1'] = [singles['time'][i[0]] for i in coins]
coinci['time2'] = [singles['time'][i[1]] for i in coins]
coinci['detector1'] = [singles['detector'][i[0]] for i in coins]
coinci['detector2'] = [singles['detector'][i[1]] for i in coins]
coinci['source1'] = [singles['source'][i[0]] for i in coins]
coinci['source2'] = [singles['source'][i[1]] for i in coins]
coinci['true'] = coinci['source1'] == coinci['source2']

In [7]:
len(coinci)

181782

In [8]:
coin_store = coincidences.copy()

In [9]:
coincidences = coinci.copy()

In [10]:
remote_file.close()

In [11]:
# System-Wide Equation Constants

S = len(singles) / total_time # Rate of singles measured by scanner as a whole
P = 2 * len(coincidences) / total_time # Twice the prompts rate

# Roots of this function are the lambda (L) values.
def lambda_eq(L):
    return 2 * TAU * L * L - L + S - P * np.exp((L + S)*TAU)

L = root_scalar(lambda_eq, x0=0)
if not L.converged:
    raise RuntimeError("Failed to converge on lambda.")
L = L.root

det1_counts = coincidences['detector1'].value_counts().to_dict() # det1 coincidences involved
det2_counts = coincidences['detector2'].value_counts().to_dict() # det2 coincidences involved

prompts = pd.DataFrame({'detector': list(range(num_detectors))})
prompts['prompts'] = prompts['detector'].map(lambda x: det1_counts.get(x, 0) + det2_counts.get(x, 0))
prompt_count = prompts.set_index('detector')['prompts'].to_dict()
singles_counts = singles['detector'].value_counts().to_dict()

In [ ]:
# Returns the randoms rate from a pair of detectors with crystalIDs i and j
def randomsrate(i, j):
    P_i = prompt_count.get(i, 0) / total_time
    P_j = prompt_count.get(j, 0) / total_time
    S_i = singles_counts.get(i, 0) / total_time
    S_j = singles_counts.get(j, 0) / total_time
    coeff = (2 * TAU * np.exp(-(L + S)*TAU))/((1 - 2 * L * TAU)**2)
    i_term = S_i - np.exp((L + S)*TAU) * P_i
    j_term = S_j - np.exp((L + S)*TAU) * P_j
    return coeff * i_term * j_term

In [13]:
sp_estimate = 0
for i in range(num_detectors):
    for j in range(i, num_detectors):
        sp_estimate += randomsrate(i, j)

print(sp_estimate)

32627.748979449585


In [83]:
sp_estimate = sp_estimate * total_time

In [84]:
print(sp_estimate)

32627.748979449585


In [85]:
actual = coincidences[coincidences['true'] == False]

In [86]:
len(actual)

35444

In [87]:
dw_estimate = 0
for t in singles['time']:
    dw_estimate += np.searchsorted(singles['time'], t + DELAY + TAU) - np.searchsorted(singles['time'], t + DELAY)
print(dw_estimate)

41606


In [88]:
sr_estimate = 0
for i in range(num_detectors):
    for j in range(i, num_detectors):
        sr_estimate += 2 * TAU * singles_counts.get(i, 0) / total_time * singles_counts.get(j, 0) / total_time

In [89]:
print(sr_estimate * total_time)

41741.50412241712
